# Языковое моделирование

## Доставляем и импортируем библиотеки

In [1]:
!pip install tensorflow

You should consider upgrading via the 'c:\users\vlad\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [2]:
import tensorflow as tf

import numpy as np
import os

## Дополнительные функции

In [3]:
def split_input_target(chunk):
    """
        Функция для разбиение на таргеты
    """
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

def loss(labels, logits):
    """
        Функция для получения функции потерь
    """
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    """
        Функция для создания модели
    """
    return tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])

def generate_text(model, temperature, start_string):
    """
        Функция для генерации текста
    """
    num_generate = 500

    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])

    return start_string + ''.join(text_generated)


## Скачиваем и подготавливаем данные

In [4]:
!wget -O 'data/evgenyi_onegin.txt' -qq --no-check-certificate https://gbcdn.mrgcdn.ru/uploads/asset/2801033/attachment/55bca7786254f79f79c4bfc6229cab4b.txt

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
text = open('data/evgenyi_onegin.txt', 'rb').read().decode(encoding='utf-8')
print('Length of text: {} characters'.format(len(text)))

Length of text: 286984 characters


In [6]:
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

131 unique characters


In [7]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [8]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [9]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '
'      Роман в стихах\n\n                        Не мысля гордый свет забавить,\n                        '
'Вниманье дружбы возлюбя,\n                        Хотел бы я тебе представить\n                        '
'Залог достойнее тебя,\n                        Достойнее души прекрасной,\n                        Свят'
'ой исполненной мечты,\n                        Поэзии живой и ясной,\n                        Высоких д'


In [10]:
dataset = sequences.map(tf.autograph.experimental.do_not_convert(split_input_target))

In [11]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                         '
Target data: 'лександр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '


In [12]:
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int32, name=None), TensorSpec(shape=(64, 100), dtype=tf.int32, name=None))>

## Обучаем модель

In [13]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 1024

In [14]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 128)           16768     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          4722688   
                                                                 
 lstm_1 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_2 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_3 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 dense (Dense)               (64, None, 131)           134275    
                                                                 
Total params: 30,051,843
Trainable params: 30,051,843
No

In [16]:
model.compile(optimizer='adam', loss=loss)

In [17]:
checkpoint_dir = 'training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=264,
    save_weights_only=True)

In [18]:
EPOCHS = 10

In [19]:
%%time
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
44/44 [==============================] - 663s 15s/step - loss: 2.4695

In [20]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [21]:
gen_text = generate_text(model, 1, start_string=u"В некотором царстве")
print(gen_text)

В некотором царствей,                          Интл сесщдушиг ра вошак
                       Ичежлен селтяхоик  
                       Им мреТиле вобижь луслосе
  Ет,йканос

                      Тро дево с сслитоти т дсмь
         Точ гик,кошелазра вега;
                          ИIдiдт вав тве срамудо
                          -Vжактнр рехднечри гне
                    Тсттычьемегырялей.к  чьянуд  росонитьныи
                           
Ткортаи лилинан шоосжокй

                      Ссчототкимлу:  пре  моику 


In [22]:
gen_text = generate_text(model, 0.8, start_string=u"Жили были")
print(gen_text)

Жили были                                                    Гос 
                       Воирнюх жщенали сборас,
                        Инуторы зымриннак ни дастоди 
                          зоч,ром лолрь,-козртдак.яи-
                         Кделоропе нке нияноляна
                                         XЗротим чоюм ввогев мрайльти

                           Вел дисокег вотений о                         Оста мры влелытибьтрет неде,
           В,
                                                    


**ВЫВОДЫ:**

Были попытки обучить модель до 100 эпох, но качество не сильно изменялось уже на 10-ой эпохе и очень много времени затрачивается на обучение, что ни есть хорошо, хотя аидны разметки текста.